## Pair selection

This Jupyter Notebook to query the catalog for a pair of Sentinel-1 SLC products

* First do the imports of the Python libraries required

In [1]:
import os
import lxml.etree as etree
import cioppy
from datetime import datetime, timedelta 
import dateutil.parser 
from shapely.wkt import loads 


from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import nbformat as nbf

import geopandas as gp

import numpy as np
import ipyleaflet
from shapely.geometry import box
from shapely.geometry import multipolygon
from shapely.wkt import loads
from ipyleaflet import Map, Polygon

* Define the search parameters: the catalog series OpenSearch endpoint, the time of interest and the area of interest

In [2]:
series = 'https://catalog.terradue.com/sentinel1/search'

s1_prd_type = 'SLC'

crop_wkt = 'POLYGON((9.789267467315815 13.545328656445236,9.789267467315815 13.293608039872737,9.963279058576214 13.289265722304478,9.952124469392855 13.538821973353123,9.789267467315815 13.545328656445236,9.789267467315815 13.545328656445236))'


### Search for a master

In [3]:
slave_self = 'https://catalog.terradue.com/sentinel1/search?uid=S1B_S1_SLC__1SDV_20180907T172154_20180907T172223_012612_017456_AB33'
aoi = loads(crop_wkt)
            
            

In [4]:
ciop = cioppy.Cioppy()
# get the slave metadata 
slave_search = ciop.search(end_point = slave_self,
                                   params = [],
                                   output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                                   model='EOP')

In [5]:
slave = slave_search[0]
        
# look for a master up to one cycle in the past
master_search_start_date = (dateutil.parser.parse(slave['startdate']) + timedelta(days=-12)).isoformat()
master_search_stop_date = (dateutil.parser.parse(slave['startdate']) + timedelta(days=-1)).isoformat()
                
# do the catalogue search    
master_search_params = dict([('geom', slave['wkt']),
                                     ('track', slave['track']),
                                     ('pt',slave['productType']),
                                     ('start', master_search_start_date),
                                     ('stop', master_search_stop_date),
                                     ('q', 'sensorOperationalMode.text:S*')])
        
series = 'https://catalog.terradue.com/sentinel1/search'

master_search = ciop.search(end_point=series, 
                                   params=master_search_params,
                                   output_fields='identifier,enclosure,wkt,self,startdate',
                                   model='EOP')
        
result = []

# build a data frame 
for index, elem in enumerate(master_search):
                 
    master_wkt = loads(elem['wkt'])
    
    result.append({'self' : elem['self'],
                           'identifier' : elem['identifier'],
                           'enclosure' : elem['enclosure'],
                           'wkt': loads(elem['wkt']),
                           'aoi_intersec' : (master_wkt.intersection(aoi).area/aoi.area) * 100,
                           'slave_intersec' : (master_wkt.intersection(loads(slave['wkt']))).area / loads(slave['wkt']).area * 100,
                           'contains': master_wkt.contains(aoi),
                           'days': (dateutil.parser.parse(slave['startdate'])- dateutil.parser.parse(elem['startdate'])).days
                          })

masters = gp.GeoDataFrame(result)

In [6]:
masters

,aoi_intersec,contains,days,enclosure,identifier,self,slave_intersec,wkt
0,0.0,False,11,https://scihub.copernicus.eu/apihub/odata/v1/P...,S1B_S1_SLC__1SDV_20180826T172222_20180826T1722...,https://catalog.terradue.com/sentinel1/search?...,0.000000,"POLYGON ((9.313912 15.268483, 10.046239 15.413..."
1,100.0,True,12,https://scihub.copernicus.eu/apihub/odata/v1/P...,S1B_S1_SLC__1SDV_20180826T172154_20180826T1722...,https://catalog.terradue.com/sentinel1/search?...,99.951386,"POLYGON ((9.521246 14.294063, 10.250069 14.439..."


In [12]:
# select the master 
master = master_search[masters.sort_values(['aoi_intersec', 'days'], 
                                                   ascending=[False, False]).iloc[0].name]
print master['identifier']

S1B_S1_SLC__1SDV_20180826T172154_20180826T172222_012437_016EEF_6AFD


In [8]:
print slave['identifier']

S1B_S1_SLC__1SDV_20180907T172154_20180907T172223_012612_017456_AB33


In [9]:
source = '%s,%s' % (master['self'],
                   slave['self'])

In [11]:
source

'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_S1_SLC__1SDV_20180826T172154_20180826T172222_012437_016EEF_6AFD,https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_S1_SLC__1SDV_20180907T172154_20180907T172223_012612_017456_AB33'